# Using Client Method

## Instance

### Create Instance

In [4]:
import boto3

# Create EC2 client
ec2 = boto3.client('ec2', region_name='us-east-1')

# Create instance
response = ec2.run_instances(
    ImageId='ami-0f3caa1cf4417e51b',        # Replace with valid AMI ID
    InstanceType='t3.micro',
    KeyName='Learning Key Pair (pem)',      # Replace with your key pair name
    MinCount=1,
    MaxCount=1,
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'py-script'
                }
            ]
        }
    ]
)

print("Instance Created!")
print("Instance ID:", response['Instances'][0]['InstanceId'])

Instance Created!
Instance ID: i-03e25f8bf8b7c26c9


### Delete/Terminate Instance

In [ ]:
import boto3

ec2 = boto3.resource('ec2', region_name='us-east-1')

instance = ec2.Instance('i-03e25f8bf8b7c26c9')

instance.terminate()
instance.wait_until_terminated()

print("EC2 Instance Deleted")

EC2 Instance Deleted


## AMI (Amazon Machine Image)

### Create AMI

In [5]:
import boto3

ec2 = boto3.client('ec2', region_name='us-east-1')

response = ec2.create_image(
    InstanceId='i-03e25f8bf8b7c26c9',
    Name='AMI-py-script',
    Description='Created via SDK (boto3)',
    NoReboot=True
)

ami_id = response['ImageId']
print("Creating AMI:", ami_id)

waiter = ec2.get_waiter('image_available')
waiter.wait(ImageIds=[ami_id])

print("AMI is available:", ami_id)

Creating AMI: ami-0105b3dd04cd653fe
AMI is available: ami-0105b3dd04cd653fe


### Delete AMI

In [2]:
import boto3

region = "us-east-1"
ami_id = "ami-0105b3dd04cd653fe"

ec2 = boto3.client('ec2', region_name=region)

# Step 1: Get snapshot IDs
image = ec2.describe_images(ImageIds=[ami_id])
snapshots = []

for bd in image['Images'][0]['BlockDeviceMappings']:
    if 'Ebs' in bd:
        snapshots.append(bd['Ebs']['SnapshotId'])

# Step 2: Deregister AMI
ec2.deregister_image(ImageId=ami_id)
print("AMI deregistered")

# Step 3: Delete snapshots
for snap in snapshots:
    ec2.delete_snapshot(SnapshotId=snap)
    print("Deleted snapshot:", snap)

AMI deregistered
Deleted snapshot: snap-09586ce23a72784ee


## Key Pair

### Create Key Pair

In [ ]:
import boto3

ec2 = boto3.client('ec2', region_name='us-east-1')
response = ec2.create_key_pair(KeyName='boto3')

private_key = response['KeyMaterial']

# Save private key to file
with open('boto3.pem', 'w') as file:
    file.write(private_key)

print("Key Pair Created Successfully")

Key Pair Created Successfully


### List all Key Pairs

In [11]:
import boto3

ec2 = boto3.client('ec2', region_name='us-east-1')

response = ec2.describe_key_pairs()
for key in response['KeyPairs']:
    print(key['KeyName'])

Learning Key Pair (ppk)
boto3
Learning Key Pair (pem)


### Delete Key Pair

In [12]:
import boto3

ec2 = boto3.client('ec2', region_name='us-east-1')
ec2.delete_key_pair(KeyName='boto3')

{'Return': True,
 'KeyPairId': 'key-04194a81f14002447',
 'ResponseMetadata': {'RequestId': 'abb0395b-39ae-4acd-a621-9b15f7ea1149',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'abb0395b-39ae-4acd-a621-9b15f7ea1149',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '259',
   'date': 'Wed, 25 Feb 2026 07:22:26 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

## Error Handling

In [10]:
import boto3
from botocore.exceptions import ClientError

ec2 = boto3.client('ec2', region_name='us-east-1')

try:
    response = ec2.create_key_pair(KeyName='boto3')
    print("Key pair created!")

    # Save private key to file (important)
    with open("boto3.pem", "w") as file:
        file.write(response['KeyMaterial'])

except ClientError as e:
    if e.response['Error']['Code'] == 'InvalidKeyPair.Duplicate':
        print("Key already exists!")
    else:
        print("Error:", e)

Key already exists!


# Using Resource Method

## Instance

### Create Instance

In [ ]:
import boto3

ec2 = boto3.resource('ec2', region_name='us-east-1')

instances = ec2.create_instances(
    ImageId='ami-0f3caa1cf4417e51b',
    MinCount=1,
    MaxCount=1,
    InstanceType='t3.micro',
    KeyName='Learning Key Pair (pem)'
)

for instance in instances:
    print("Instance ID:", instance.id)

Instance ID: i-0f5c05a53404bd713


### Delete/Terminate Instance

In [ ]:
import boto3

ec2 = boto3.resource('ec2', region_name='us-east-1')

instance = ec2.Instance('i-0f5c05a53404bd713')

instance.terminate()
instance.wait_until_terminated()

print("EC2 Instance Deleted")

EC2 Instance Deleted


## Key Pair

### Create Key Pair

In [ ]:
import boto3

ec2 = boto3.resource('ec2', region_name='us-east-1')

key_pair = ec2.create_key_pair(KeyName='boto3')

with open('boto3.pem', 'w') as file:
    file.write(key_pair.key_material)

print("Key Created:", key_pair.name)

Key Created: boto3


### Delete Key Pair

In [7]:
import boto3

ec2 = boto3.resource('ec2', region_name='us-east-1')

# Get key pair resource
key_pair = ec2.KeyPair('boto3')

# Delete key pair
key_pair.delete()

print("Key pair deleted:", key_pair.name)

Key pair deleted: boto3
